In [78]:
# Importar SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, desc, rank, col, dense_rank, max
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as spark_sum

# Importações para exibição dos gráficos
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Importações para criar o dashboard
import dash
from dash import dcc, html
import plotly.graph_objs as go


In [79]:
# Inicializar SparkSession

spark = SparkSession.builder \
    .appName("Produtos Mais Importados e Exportados") \
    .getOrCreate()

#### Caminhos dos arquivos

In [80]:
# Tabela de exportações
caminho_exportacoes = "EXP_COMPLETA.csv"

# Tabela de importações
caminho_importacoes ="IMP_COMPLETA.csv" 

# Tabela de nomeclatura dos nomes, colunas e produtos.
caminho_tabela_produtos = "NCM.csv"


## Carregando o primeiro dataframe relacionado às exportações

In [81]:

df_exp = (spark
            .read
            .format('csv')
            .option('header','true')
            .option('inferSchema','true')
            .option('index','true')             
            .option('delimiter',';')
            .load(caminho_exportacoes))


## Carregando o segundo dataframe relacionado às Importações

In [82]:

df_imp = (spark
            .read
            .format('csv')
            .option('header','true')
            .option('inferSchema','true')
            .option('index','true')             
            .option('delimiter',';')
            .load(caminho_importacoes))


## Carregando o dataframe relacionado a nomeclatura das colunas

In [83]:
df_table = (spark
            .read
            .format('csv')
            .option('header','true')
            .option('inferSchema','true')
            .option('index','true')             
            .option('delimiter',';')
            .load(caminho_tabela_produtos))


### Excluindo colunas no dataframe

In [84]:
colunas_para_excluir = ["NO_NCM_ESP", "NO_NCM_POR"]
new_df = df_table
for coluna in colunas_para_excluir:
    new_df = new_df.drop(coluna)

In [85]:
# dataframe somente com a coluna NO_NCM_ING
'''new_df.show()'''

'new_df.show()'

### Adicionando a coluna com os nomes dos produtos na tabela de EXPORTAÇÃO e tabela de IMPORTAÇÃO

In [86]:
# Tabela de Exportação
df_com_produtos = df_exp.join(df_table.select("CO_NCM", "NO_NCM_ING"), "CO_NCM", "left_outer")

# Tabela de Importação
df_imp_com_produtos = df_imp.join(df_table.select("CO_NCM", "NO_NCM_ING"), "CO_NCM", "left_outer")

In [87]:
'''df_com_produtos.show()'''
'''df_imp_com_produtos.show()'''

'df_imp_com_produtos.show()'

## 01. Mostre qual os top 3 produtos mais exportados em 2017, 2018 e 2019 ;

### Filtrar dados para os anos de interesse (2017, 2018, 2019)

In [88]:

df_ano = df_com_produtos.filter(df_com_produtos["CO_ANO"].isin(2017, 2018, 2019))
'''df_ano.show()'''

'df_ano.show()'

### Agrupar por estado, somando as quantidades exportadas

In [89]:

df_agrupado_estado= df_ano.groupBy("CO_ANO", "CO_NCM","SG_UF_NCM", "NO_NCM_ING").max("QT_ESTAT").withColumnRenamed("max(QT_ESTAT)", "TOTAL_EXPORTADO")
'''df_agrupado_estado.show()'''


'df_agrupado_estado.show()'

### Selecionar o maior valor de exportação por ano

In [90]:

df_max_exportacao = df_agrupado_estado.groupBy("CO_ANO","CO_NCM","SG_UF_NCM","NO_NCM_ING").agg(max("TOTAL_EXPORTADO").alias("MAX_EXPORTACAO"))
'''df_max_exportacao.show()'''

'df_max_exportacao.show()'

### Aplicar a função dense_rank para identificar os 3 maiores valores de "MAX_EXPORTACAO"

In [91]:

df_max_exportacao_com_rank = df_max_exportacao.withColumn("rank", dense_rank().over(Window.orderBy(col("MAX_EXPORTACAO").desc())))

In [92]:
'''df_max_exportacao_com_rank.show()'''

'df_max_exportacao_com_rank.show()'

### Selecionar apenas as linhas com rank <= 3 (menor ou igual a três) para obter os 3 top produtos 

In [93]:

tres_maiores = df_max_exportacao_com_rank.filter(col("rank") <= 3)

# Neste resultado temos o top 3 produtos mais exportados por estado e ano.
'''tres_maiores.show()'''

'tres_maiores.show()'

In [94]:
# Selecionar apenas as colunas desejados do df
df_selecionado = tres_maiores.select("CO_ANO", "SG_UF_NCM", "MAX_EXPORTACAO","NO_NCM_ING")

# Convertendo para um dt pandas
df1_pandas = df_selecionado.toPandas()
'''df1_pandas'''

'df1_pandas'

### Construindo um gráfico sobre "Os top 3 produtos mais exportados por estado nos anos de 2017, 2018 e 2019"

In [95]:
# Encontrar o produto principal (com a maior quantidade exportada)
produto_principal = df1_pandas.groupby('NO_NCM_ING')['MAX_EXPORTACAO'].sum().idxmax()

# Criar uma lista de cores para cada ano
cores = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 'rgb(44, 160, 44)']

# Criar uma lista de fatias para o gráfico de pizza
fatias = []

# Iterar sobre cada estado
for estado in df1_pandas['SG_UF_NCM'].unique():
    # Filtrar o DataFrame para o estado atual
    df_estado = df1_pandas[df1_pandas['SG_UF_NCM'] == estado]
    
    # Criar uma lista de valores de exportação para o estado atual
    valores_exportacao = []
    
    # Iterar sobre cada ano
    for ano, cor in zip(df1_pandas['CO_ANO'].unique(), cores):
        # Filtrar o DataFrame para o ano atual
        df_ano = df_estado[df_estado['CO_ANO'] == ano]
        
        # Calcular a quantidade total exportada pelo estado atual no ano atual
        total_ano = df_ano['MAX_EXPORTACAO'].sum()
        valores_exportacao.append(total_ano)
    
    fatias.append(go.Pie(
        labels=[f'{estado} - {produto_principal} - {ano}' for ano in df1_pandas['CO_ANO'].unique()],  # Estado, produto e ano
        values=valores_exportacao,  # Quantidades exportadas por ano
        hoverinfo='label+percent+value',  # Informações exibidas ao passar o mouse
        marker=dict(colors=cores),  # Cores das fatias
    ))

# Criar o layout do gráfico
layout = go.Layout(
    title='Quantidade Exportada por Estado e Ano',  # Título do gráfico
    legend_title='Estado - Produto - Ano',  # Título da legenda
)

# Criar a figura do gráfico
fig1 = go.Figure(data=fatias, layout=layout)

# Exibir o gráfico
'''fig1.show()'''

'fig1.show()'

## 02. Mostre qual os top 3 produtos mais importados em 2017, 2018 e 2019;

In [96]:
# filtrar os anos 
df_ano_imp = df_imp_com_produtos.filter(df_imp["CO_ANO"].isin(2017, 2018, 2019))
'''df_ano_imp.show()'''

'df_ano_imp.show()'

### Agrupar por estado, somando as quantidades importadas

In [97]:
df_agrupado_estado_imp= df_ano_imp.groupBy("CO_ANO", "CO_NCM","SG_UF_NCM", "NO_NCM_ING").max("QT_ESTAT").withColumnRenamed("max(QT_ESTAT)", "MAX_IMPORTACAO")
'''df_agrupado_estado_imp.show()'''

'df_agrupado_estado_imp.show()'

### Aplicar a função dense_rank para identificar os maiores valores de "MAX_IMPORTAÇÃO"

In [98]:
df_max_importacao_com_rank = df_agrupado_estado_imp.withColumn("rank", dense_rank().over(Window.orderBy(col("MAX_IMPORTACAO").desc())))
'''df_max_importacao_com_rank.show()'''

'df_max_importacao_com_rank.show()'

### Selecionar os tops 3 produtos 

In [99]:
# Selecionar registros com ranks específicos
ranks_especificos = [1, 4, 8]
df_top_3_imp = df_max_importacao_com_rank.filter(col("rank").isin(ranks_especificos))
'''df_top_3_imp.show()'''

'df_top_3_imp.show()'

In [100]:
# Selecionar apenas as colunas desejados do df
df_top3_imp = df_top_3_imp.select("CO_ANO", "SG_UF_NCM", "MAX_IMPORTACAO","NO_NCM_ING")

# Convertendo para um dataframe pandas para exibição de gráficos  
df2_pandas = df_top3_imp.toPandas()
'''df2_pandas'''

'df2_pandas'

### Construindo um gráfico sobre "Os top 3 produtos mais Importados por estado nos anos de 2017, 2018 e 2019"

In [101]:
# Criar o gráfico de barras empilhadas
fig2 = go.Figure()

# Adicionar barras para cada estado e produto
for estado in df2_pandas['SG_UF_NCM'].unique():
    df_estado = df2_pandas[df2_pandas['SG_UF_NCM'] == estado]
    for ano in df_estado['CO_ANO'].unique():
        df_ano = df_estado[df_estado['CO_ANO'] == ano]
        fig2.add_trace(go.Bar(
            x=df_ano['NO_NCM_ING'],  # Produtos no eixo x
            y=df_ano['MAX_IMPORTACAO'],  # Importações máximas no eixo y
            name=f'{estado} - {ano} ',  # Nome do estado e ano
            hoverinfo='name+y'  # Informações exibidas ao passar o mouse
        ))

# Personalizar o layout do gráfico
fig2.update_layout(
    title='Importações Máximas por Produto, Estado e Ano',
    xaxis_title='Produto',
    yaxis_title='Importação Máxima',
    barmode='stack',  # Barras empilhadas
    xaxis={'categoryorder': 'total descending'}, # Ordenar os produtos pela soma das importações máximas
    yaxis=dict(title='Importação Máxima', zeroline=False),
    legend_title='Estado - Ano'  # Título da legenda
)

# Exibir o gráfico
'''fig2.show()'''

'fig2.show()'

## 03. Mostre qual os top 3 produtos exportados em cada mês de 2019 por estado;

In [102]:
# filtrar os ano de 2019
df_ano_exp = df_com_produtos.filter(df_com_produtos["CO_ANO"].isin(2019))
'''df_ano_exp.show()'''

'df_ano_exp.show()'

### agrupar por estado

In [103]:

df_agrupado_estado_exp19= df_ano_exp.groupBy("CO_ANO", "CO_NCM","SG_UF_NCM", "NO_NCM_ING").max("QT_ESTAT").withColumnRenamed("max(QT_ESTAT)", "MAX_EXPORTACAO")

### Ordenando os anos em ordem crescente de acordo com sua exportação

In [104]:

df_max_importacao19_com_rank = df_agrupado_estado_exp19.withColumn("rank", dense_rank().over(Window.orderBy(col("MAX_EXPORTACAO").desc())))

### Filtrando os 3 maiores valores

In [105]:
df3_tres_maiores = df_max_importacao19_com_rank.filter(col("rank") <= 3)

In [106]:
# Selecionar apenas as colunas desejados do dataframe
df_top19_exp = df3_tres_maiores.select("CO_ANO", "CO_NCM" ,"SG_UF_NCM", "NO_NCM_ING", "MAX_EXPORTACAO")

# Convertendo para um dataframe pandas para exibição de gráficos  
df3_pandas = df_top19_exp.toPandas()
'''df3_pandas'''

'df3_pandas'

### Construindo um gráfico sobre "Os top 3 produtos mais Exportados em cada mês no ano de 2019"

In [107]:
# Criar o gráfico de barras empilhadas
fig3 = go.Figure()

# Adicionar barras para cada estado e produto
for estado in df3_pandas['SG_UF_NCM'].unique():
    df_estado = df3_pandas[df3_pandas['SG_UF_NCM'] == estado]
    for ano in df_estado['CO_ANO'].unique():
        df_ano = df_estado[df_estado['CO_ANO'] == ano]
        fig3.add_trace(go.Bar(
            x=df_ano['SG_UF_NCM'],  # Produtos no eixo x
            y=df_ano['MAX_EXPORTACAO'],  # Importações máximas no eixo y
            name=f'{estado} - {ano} - {df_ano["NO_NCM_ING"].iloc[0]}',  # Nome do estado, ano e produto
            hoverinfo='name+y'  # Informações exibidas ao passar o mouse
        ))

# Personalizar o layout do gráfico
fig3.update_layout(
    title='Exportações Máximas por estado em 2019',
    xaxis_title='Estado',
    yaxis_title='Exportação Máxima',
    barmode='stack',  # Barras empilhadas
    xaxis={'categoryorder': 'total descending'}, # Ordenar os produtos pela soma das importações máximas
    yaxis=dict(title='Exportação Máxima', zeroline=False),
    legend_title='Estado - Ano - Produto'  # Título da legenda
)

# Exibir o gráfico
'''fig3.show()'''

'fig3.show()'

## 04. Representatividade em valor de exportação por estado no ano de 2019 em relação ao total exportado pelo país no mesmo ano;

In [108]:
# Filtrar os dados apenas para o ano de 2019
df4_ano_exp = df_com_produtos.filter(df_com_produtos['CO_ANO'] == 2019)

### Agrupar e somar a quantidade exportada por cada estado no ano de 2019

In [109]:
total_exportado_por_estado2019 = df4_ano_exp.groupBy("CO_ANO","SG_UF_NCM").sum("QT_ESTAT").withColumnRenamed("sum(QT_ESTAT)", "MAX_EXPORTACAO_ESTADO")

In [110]:
total_exportado_por_estado2019.show()

+------+---------+---------------------+
|CO_ANO|SG_UF_NCM|MAX_EXPORTACAO_ESTADO|
+------+---------+---------------------+
|  2019|       RO|            600046018|
|  2019|       AL|             36032122|
|  2019|       GO|           7336197426|
|  2019|       DF|             43000596|
|  2019|       PI|            164853846|
|  2019|       ES|          27213531364|
|  2019|       AM|           1000183885|
|  2019|       PR|          10841681756|
|  2019|       TO|            546172642|
|  2019|       PA|         195801879502|
|  2019|       MT|          25647509292|
|  2019|       BA|           4418664214|
|  2019|       MA|           9368840695|
|  2019|       RJ|          21211098313|
|  2019|       RN|           1142299073|
|  2019|       RR|             84939937|
|  2019|       ND|           3212965829|
|  2019|       PB|             89366873|
|  2019|       AC|             27453591|
|  2019|       AP|            500578595|
+------+---------+---------------------+
only showing top

### Somar a quantidade total exportada pelo país no ano de 2019

In [111]:
total_exportado_em_2019 = total_exportado_por_estado2019.groupBy("CO_ANO").sum("MAX_EXPORTACAO_ESTADO").withColumnRenamed("sum(MAX_EXPORTACAO_ESTADO)", "TOTAL_EXPORTADO_2019")

In [112]:
total_exportado_em_2019.show()

+------+--------------------+
|CO_ANO|TOTAL_EXPORTADO_2019|
+------+--------------------+
|  2019|        526083644677|
+------+--------------------+



### fazer a conversão dos dois dataframes spark para pandas

In [113]:
# Exportações por estado

df4_sum_estado_pandas = total_exportado_por_estado2019.toPandas()

# Exportações por ano

df4_sum_anual_pandas = total_exportado_em_2019.toPandas()

### Construindo um gráfico sobre  a "Representatividade em valor de EXPORTAÇÃO por estado no ano de 2019 em relação ao total exportado pelo país no mesmo ano"

In [114]:
# Extrair os dados das exportações por estado
estados = df4_sum_estado_pandas['SG_UF_NCM']
exportacoes_por_estado = df4_sum_estado_pandas['MAX_EXPORTACAO_ESTADO']

# Extrair a exportação total de 2019
exportacao_total_2019 = df4_sum_anual_pandas['TOTAL_EXPORTADO_2019'].iloc[0]

# Criar o gráfico de barras
fig4 = go.Figure()

# Adicionar barras para as exportações por estado
fig4.add_trace(go.Bar(
    x=estados,
    y=exportacoes_por_estado,
    name='Exportação por Estado'
))

# Adicionar uma linha para a exportação total de 2019
fig4.add_trace(go.Scatter(
    x=estados,
    y=[exportacao_total_2019] * len(estados),
    mode='lines',
    name='Exportação Total de 2019',
    line=dict(color='red', dash='dash')
))

# Personalizar o layout do gráfico
fig4.update_layout(
    title='Comparação das Exportações por Estado com a Exportação Total de 2019',
    xaxis_title='Estado',
    yaxis_title='Valor de Exportação',
    barmode='group',
    showlegend=True
)

# Exibir o gráfico
'''fig4.show()'''

'fig4.show()'

## 05. Representatividade em valor de importação por estado no ano de 2019 em relação ao total importado pelo país no mesmo ano.

In [115]:
# Filtrar os dados apenas para o ano de 2019

df5_ano_imp = df_imp_com_produtos.filter(df_imp_com_produtos['CO_ANO'] == 2019)

### Agrupar e somar a quantidade importada por cada estado no ano de 2019

In [116]:
total_importado_por_estado2019 = df5_ano_imp.groupBy("CO_ANO","SG_UF_NCM").sum("QT_ESTAT").withColumnRenamed("sum(QT_ESTAT)", "MAX_IMPORTACAO_ESTADO")
'''total_importado_por_estado2019.show()'''

'total_importado_por_estado2019.show()'

### Somar a quantidade total importada pelo país no ano de 2019

In [117]:
total_importado_em_2019 = total_importado_por_estado2019.groupBy("CO_ANO").sum("MAX_IMPORTACAO_ESTADO").withColumnRenamed("sum(MAX_IMPORTACAO_ESTADO)", "TOTAL_IMPORTADO_2019")

In [118]:
total_importado_em_2019.show()

+------+--------------------+
|CO_ANO|TOTAL_IMPORTADO_2019|
+------+--------------------+
|  2019|        238234076545|
+------+--------------------+



### fazer a conversão dos dois dataframes spark para pandas

In [119]:
# Importações por estado

df5_sum_estado_pandas = total_importado_por_estado2019.toPandas()

# Importações por ano

df5_sum_anual_pandas = total_importado_em_2019.toPandas()

### Construindo um gráfico sobre a "Representatividade em valor de IMPORTAÇÃO por estado no ano de 2019 em relação ao total importado pelo país no mesmo ano."

In [120]:

estados_imp = df5_sum_estado_pandas['SG_UF_NCM']
importacoes_por_estado = df5_sum_estado_pandas['MAX_IMPORTACAO_ESTADO']

# Extrair a exportação total de 2019
importacao_total_2019 = df5_sum_anual_pandas['TOTAL_IMPORTADO_2019'].iloc[0]

# Criar o gráfico de barras
fig5 = go.Figure()

# Adicionar barras para as exportações por estado
fig5.add_trace(go.Bar(
    x=estados_imp,
    y=importacoes_por_estado,
    name='Importação por Estado'
))

# Adicionar uma linha para a exportação total de 2019
fig5.add_trace(go.Scatter(
    x=estados_imp,
    y=[importacao_total_2019] * len(estados_imp),
    mode='lines',
    name='Importação Total de 2019',
    line=dict(color='red', dash='dash')
))

# Personalizar o layout do gráfico
fig5.update_layout(
    title='Comparação das Importações por Estado com a Importação Total de 2019',
    xaxis_title='Estado',
    yaxis_title='Valor de Importação',
    barmode='group',
    showlegend=True
)

# Exibir o gráfico
'''fig5.show()'''

'fig5.show()'

# testes do dashboard


In [121]:
# Inicialize o aplicativo Dash
app = dash.Dash(__name__)

# Layout do aplicativo
app.layout = html.Div(children=[
    html.H1(children='Meu Dashboard'),

    html.Div(children='''
        Gráfico de Quantidade Exportada por Estado e Ano
    '''),

    # Adicione o primeiro gráfico como um componente dcc.Graph
    dcc.Graph(
        id='grafico-1',
        figure={
            'data': [
                go.Pie(
                    labels=[f'{estado} - {produto_principal} - {ano}' for ano in df1_pandas['CO_ANO'].unique()],  # Estado, produto e ano
                    values=valores_exportacao,  # Quantidades exportadas por ano
                    hoverinfo='label+percent+value',  # Informações exibidas ao passar o mouse
                    marker=dict(colors=cores),  # Cores das fatias
                )
            ],
            'layout': {
                'title': 'Quantidade Exportada por Estado e Ano',  # Título do gráfico
                'legend_title': 'Estado - Produto - Ano'  # Título da legenda
            }
        }
    ),

    html.Div(children='''
        Gráfico de Importações Máximas por Produto, Estado e Ano
    '''),

    # Adicione o segundo gráfico como um componente dcc.Graph
    dcc.Graph(
        id='grafico-2',
        figure={
            'data': [
                go.Bar(
                    x=df_ano['NO_NCM_ING'],  # Produtos no eixo x
                    y=df_ano['MAX_IMPORTACAO'],  # Importações máximas no eixo y
                    name=f'{estado} - {ano} ',  # Nome do estado e ano
                    hoverinfo='name+y'  # Informações exibidas ao passar o mouse
                )
                for estado in df2_pandas['SG_UF_NCM'].unique()
                for ano in df2_pandas[df2_pandas['SG_UF_NCM'] == estado]['CO_ANO'].unique()
                for df_estado in [df2_pandas[df2_pandas['SG_UF_NCM'] == estado]]
                for df_ano in [df_estado[df_estado['CO_ANO'] == ano]]
            ],
            'layout': {
                'title': 'Importações Máximas por Produto, Estado e Ano',  # Título do gráfico
                'xaxis_title': 'Produto',  # Título do eixo x
                'yaxis_title': 'Importação Máxima',  # Título do eixo y
                'barmode': 'stack',  # Barras empilhadas
                'xaxis': {'categoryorder': 'total descending'},  # Ordenar os produtos pela soma das importações máximas
                'yaxis': {'title': 'Importação Máxima', 'zeroline': False},  # Título do eixo y e remoção da linha base
                'legend_title': 'Estado - Ano'  # Título da legenda
            }
        }
    ),

    html.Div(children='''
        Gráfico de Exportações Máximas por Estado em 2019
    '''),

    # Adicione o terceiro gráfico como um componente dcc.Graph
    dcc.Graph(
        id='grafico-3',
        figure={
            'data': [
                go.Bar(
                    x=df_ano['SG_UF_NCM'],  # Produtos no eixo x
                    y=df_ano['MAX_EXPORTACAO'],  # Importações máximas no eixo y
                    name=f'{estado} - {ano} - {df_ano["NO_NCM_ING"].iloc[0]}',  # Nome do estado, ano e produto
                    hoverinfo='name+y'  # Informações exibidas ao passar o mouse
                )
                for estado in df3_pandas['SG_UF_NCM'].unique()
                for ano in df3_pandas[df3_pandas['SG_UF_NCM'] == estado]['CO_ANO'].unique()
                for df_estado in [df3_pandas[df3_pandas['SG_UF_NCM'] == estado]]
                for df_ano in [df_estado[df_estado['CO_ANO'] == ano]]
            ],
            'layout': {
                'title': 'Exportações Máximas por Estado em 2019',  # Título do gráfico
                'xaxis_title': 'Estado',  # Título do eixo x
                'yaxis_title': 'Exportação Máxima',  # Título do eixo y
                'barmode': 'stack',  # Barras empilhadas
                'xaxis': {'categoryorder': 'total descending'},  # Ordenar os produtos pela soma das exportações máximas
                'yaxis': {'title': 'Exportação Máxima', 'zeroline': False},  # Título do eixo y e remoção da linha base
                'legend_title': 'Estado - Ano - Produto'  # Título da legenda
            }
        }
    ),

    html.Div(children='''
        Gráfico de Comparação das Exportações por Estado com a Exportação Total de 2019
    '''),

    # Adicione o quarto gráfico como um componente dcc.Graph
    dcc.Graph(
        id='grafico-4',
        figure={
            'data': [
                go.Bar(
                    x=estados,
                    y=exportacoes_por_estado,
                    name='Exportação por Estado'
                ),
                go.Scatter(
                    x=estados,
                    y=[exportacao_total_2019] * len(estados),
                    mode='lines',
                    name='Exportação Total de 2019',
                    line=dict(color='red', dash='dash')
                )
            ],
            'layout': {
                'title': 'Comparação das Exportações por Estado com a Exportação Total de 2019',  # Título do gráfico
                'xaxis_title': 'Estado',  # Título do eixo x
                'yaxis_title': 'Valor de Exportação',  # Título do eixo y
                'barmode': 'group',  # Agrupamento de barras
                'showlegend': True  # Mostrar legenda
            }
        }
    ),

    html.Div(children='''
        Gráfico de Comparação das Importações por Estado com a Importação Total de 2019
    '''),

    # Adicione o quinto gráfico como um componente dcc.Graph
    dcc.Graph(
        id='grafico-5',
        figure={
            'data': [
                go.Bar(
                    x=estados_imp,
                    y=importacoes_por_estado,
                    name='Importação por Estado'
                ),
                go.Scatter(
                    x=estados_imp,
                    y=[importacao_total_2019] * len(estados_imp),
                    mode='lines',
                    name='Importação Total de 2019',
                    line=dict(color='red', dash='dash')
                )
            ],
            'layout': {
                'title': 'Comparação das Importações por Estado com a Importação Total de 2019',  # Título do gráfico
                'xaxis_title': 'Estado',  # Título do eixo x
                'yaxis_title': 'Valor de Importação',  # Título do eixo y
                'barmode': 'group',  # Agrupamento de barras
                'showlegend': True  # Mostrar legenda
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)


# FIM DO DESAFIO
